In [1]:

from typing import Type, TypedDict, Union, Optional
from enum import Enum
from datetime import datetime
from pydantic import BaseModel, validator, root_validator

class BatteryManufacturer(str, Enum):  # we may want to allow custom manufacturers
    bigmap = 'BIG-MAP'
    conrad = 'Conrad energy'
    maxell = 'Maxell'

class BatteryComposition(BaseModel):
    description: str = None
    cathode: str = None
    anode: str = None
    electrolyte: str = None
    
    # check that components are not specified if string description was specified
    # build components from string description, or vice versa
    @root_validator
    def validate_composition(cls, values):
        if values['description']:
            if any([values[key] for key in ('cathode', 'anode', 'electrolyte')]):
                raise ValueError("You cannot specify a 'description' and any component at the same time.")
            components = values['description'].split('|')
            if len(components) == 3:
                values['cathode'], values['electrolyte'], values['anode'] = components
            else:
                raise ValueError("Composition 'description' does not have 3 components (i.e. {cathode}|{electrolyte}|{anode}).")
        elif any([values[key] for key in ('cathode', 'anode', 'electrolyte')]):
            values['description'] = f"{values['cathode']}|{values['electrolyte']}|{values['anode']}"
        else:
            raise ValueError("You must specify either a string 'description' or the components.")
        return values
    
class BatteryCapacity(TypedDict): # TypedDict?
    nominal: float
    actual: float

class BatteryMetadata(BaseModel):
    creation_datetime = datetime
    creation_process = str
    
class BatterySpecs(BaseModel):
    """
    Battery specification schema.
    """
    manufacturer: BatteryManufacturer
    composition: Type[BatteryComposition]
    form_factor: Union[int, str]
    capacity: Type[BatteryCapacity]
    metadata: Type[BatteryMetadata]

In [13]:
m = BatteryManufacturer.bigmap

In [56]:
BatteryComposition.schema()

{'title': 'BatteryComposition',
 'type': 'object',
 'properties': {'description': {'title': 'Description', 'type': 'string'},
  'cathode': {'title': 'Cathode', 'type': 'string'},
  'anode': {'title': 'Anode', 'type': 'string'},
  'electrolyte': {'title': 'Electrolyte', 'type': 'string'}},
 'required': ['description']}

In [75]:
if any([1, None]):
    print('yes')

yes


In [135]:
comp = BatteryComposition(anode='A')

In [134]:
comp

{'anode': 'A'}

['Lio', 'Ciao', 'oxi']

In [4]:
BatteryManufacturer

[<enum 'BatteryManufacturer'>, str, <enum 'Enum'>, object]

## CYCLING

In [2]:
# -*- coding: utf-8 -*-

from pydantic import (BaseModel, Extra, validator, validator, Field, NonNegativeFloat, NonNegativeInt)
from typing import Dict, Generic, Sequence, TypeVar, Literal, Union
from pydantic.generics import GenericModel
from aurora.schemas.cycling import CyclingTechnique, CyclingParameter, allowed_E_ranges, allowed_I_ranges

In [3]:
class ConstantVoltage(CyclingTechnique, extra=Extra.forbid):
    technique: Literal["constant_voltage"] = "constant_voltage"
    short_name: Literal["CV"] = "CV"
    name = "CV"
    description = "Controlled voltage technique, with optional current and voltage limits"
    parameters: Dict[str, CyclingParameter] = {
        "time": CyclingParameter[NonNegativeFloat](
            label = "Time:",
            description = "Maximum duration of the CV step",
            units = "s",
            value = 0.0,
            required = True,
        ),
        "voltage": CyclingParameter[float](
            label = "Step voltage:",
            description = "Voltage of the current step",
            units = "V",
            value = 0.0,
            required = True,
        ),
        "record_every_dt": CyclingParameter[NonNegativeFloat](
            label = "Record every $dt$:",
            description = "Record a datapoint at prescribed time spacing",
            units = "s",
            value = 30.0
        ),
        "record_every_dI": CyclingParameter[NonNegativeFloat](
            label = "Record every $dI$:",
            description = "Record a datapoint at prescribed current spacing",
            units = "I",
            value = 0.001
        ),
        "I_range": CyclingParameter[allowed_I_ranges](
            label = "I range",
            description = "Select the current range of the potentiostat",
            value = "keep"
        ),
        "E_range": CyclingParameter[allowed_E_ranges](
            label = "E range",
            description = "Select the voltage range of the potentiostat",
            value = "auto"
        ),
        "n_cycles": CyclingParameter[NonNegativeInt](
            label = "Number of cycles:",
            description = "Cycle through the current technique N times.",
            value = 0,
        ),
        "is_delta": CyclingParameter[bool](
            label = "$\delta V$:",
            description = "Is the step voltage a $\delta$ from previous step?",
            value = False
        ),
        "exit_on_limit": CyclingParameter[bool](
            label = "Exit when limits reached?",
            description = "Stop the whole experiment when limit is reached?",
            value = False
        ),
        "limit_voltage_max": CyclingParameter[float](
            label = "Maximum voltage:",
            description = "Define the upper limit of voltage for this step",
            value = None
        ),
        "limit_voltage_min": CyclingParameter[float](
            label = "Minimum voltage:",
            description = "Define the lower limit of voltage for this step",
            value = None
        ),
        "limit_current_max": CyclingParameter[Union[float, str]](
            label = "Maximum current:",
            description = "Define the upper limit of current for this step",
            value = None
        ),
        "limit_current_min": CyclingParameter[Union[float, str]](
            label = "Minimum current:",
            description = "Define the lower limit of current for this step",
            value = None
        )
    }

In [4]:
t = ConstantVoltage(name='CV_1')
# t1.parameters['time'].value = None

In [30]:
t.parameters

{'time': CyclingParameter[NonNegativeFloat](label='Time:', description='Maximum duration of the CV step', units='s', value=0.0, required=True),
 'voltage': CyclingParameter[float](label='Step voltage:', description='Voltage of the current step', units='V', value=0.0, required=True),
 'record_every_dt': CyclingParameter[NonNegativeFloat](label='Record every $dt$:', description='Record a datapoint at prescribed time spacing', units='s', value=30.0, required=False),
 'record_every_dI': CyclingParameter[NonNegativeFloat](label='Record every $dI$:', description='Record a datapoint at prescribed current spacing', units='I', value=0.001, required=False),
 'I_range': CyclingParameter[Literal['keep', '100 pA', '1 nA', '10 nA', '100 nA', '1 uA', '10 uA', '100 uA', '1 mA', '10 mA', '100 mA', '1 A', 'booster', 'auto']](label='I range', description='Select the current range of the potentiostat', units='', value='keep', required=False),
 'E_range': CyclingParameter[Literal['+-2.5 V', '+-5.0 V', '+-1

In [29]:
t.parameters['limit_current_max'].value = '10'